In [ ]:
import pandas as pd
df=pd.read_csv('./data/train.tsv',sep='\t')
dft=df['text'].tolist()

with open('./data/exam.txt', "w", encoding='utf-8') as f:
    for d in dft:
        line=''
        line=line+d
        f.write('{}\n'.format(line.strip()))

## 유니그램, 바이그램 자모분해

In [3]:
uni=[]
bi=[]
with open('./data/exam.txt', "r", encoding='utf-8') as infile:
    for line in infile:
        a=line.split()
        for i in a:
            uni.append(i)
        for i in range(len(a)-1):
            b=a[i]+" "+a[i+1]
            bi.append(b)
            
from collections import Counter
uni_c=Counter(uni).most_common(n=100)
bi_c=Counter(bi).most_common(n=100) 

In [4]:
with open('./data/dict_uni.txt', "w", encoding='utf-8') as f:
    for d in uni_c:
        line=''
        line=line+d[0]+" "+str(d[1])
        f.write('{}\n'.format(line.strip()))
        
with open('./data/dict_bi.txt', "w", encoding='utf-8') as f:
    for d in bi_c:
        line=''
        line=line+d[0]+" "+str(d[1])
        f.write('{}\n'.format(line.strip()))

In [5]:
from jamo import h2j, j2hcj
with open('./data/dict_uni_d.txt', "w", encoding='utf-8') as f:
    for d in uni_c:
        line=''
        line=line+j2hcj(h2j(d[0]))+" "+str(d[1])
        f.write('{}\n'.format(line.strip()))

with open('./data/dict_bi_d.txt', "w", encoding='utf-8') as f:
    for d in bi_c:
        line=''
        line=line+j2hcj(h2j(d[0]))+" "+str(d[1])
        f.write('{}\n'.format(line.strip()))

## 오타보정

In [6]:
from symspellpy_ko_master.symspellpy_ko import KoSymSpell, Verbosity

sym_spell = KoSymSpell()
sym_spell.load_korean_dictionary(decompose_korean=True, load_bigrams=True) # symspell.py에서 딕셔너리 파일 명 바꿔주자

In [7]:
text1 = "삼성젼자쥬가 올마야"
text2="듀산애너빌리티 시새는?"
text3="LG 우 종가는?"

for suggestion in sym_spell.lookup_compound(text1, max_edit_distance=2):
    a=suggestion.term
    print(text1+'  >>>  '+a)
for suggestion in sym_spell.lookup_compound(text2, max_edit_distance=2):
    b=suggestion.term
    print(text2+'  >>>  '+b)
for suggestion in sym_spell.lookup_compound(text3, max_edit_distance=2):
    c=suggestion.term
    print(text3+'  >>>  '+c)

삼성젼자쥬가 올마야  >>>  삼성전자 주가 얼마야
듀산애너빌리티 시새는?  >>>  두산에너빌리티 시세는?
LG 우 종가는?  >>>  LG우 종가는?


## 엔티티 지정

In [8]:
key=['삼성전자','두산에너빌리티','LG우','주가','시세','종가']
value=['st','st','st','pr','pr','pr']
dict_ner=dict(zip(key,value))
dict_ner

{'삼성전자': 'st',
 '두산에너빌리티': 'st',
 'LG우': 'st',
 '주가': 'pr',
 '시세': 'pr',
 '종가': 'pr'}

## 오타 보정 후 ner분석

In [9]:
import konlpy
from konlpy.tag import Komoran
komoran = Komoran(userdic='my_dict.txt') # 코모란 사전 이걸로 교체
words1 = komoran.pos(a)
words2 = komoran.pos(b)
words3 = komoran.pos(c)
nouns1=[w[0] for w in words1 if ('NNG' in w or 'NNP' in w)]
nouns2=[w[0] for w in words2 if ('NNG' in w or 'NNP' in w)]
nouns3=[w[0] for w in words3 if ('NNG' in w or 'NNP' in w)]
ner1=[(i,dict_ner[i]) for i in nouns1 if (i in dict_ner)]
ner2=[(i,dict_ner[i]) for i in nouns2 if (i in dict_ner)]
ner3=[(i,dict_ner[i]) for i in nouns3 if (i in dict_ner)]
print(text1,'  >>>  ',a,'  >>>  ','Intent:',ner1)
print(text2,'  >>>  ',b,'  >>>  ','Intent:',ner2)
print(text3,'  >>>  ',c,'  >>>  ','Intent:',ner3)  # LG우 는 정식 종목명이 아니라 형태소 사전에 없음 

삼성젼자쥬가 올마야   >>>   삼성전자 주가 얼마야   >>>   Intent: [('삼성전자', 'st'), ('주가', 'pr')]
듀산애너빌리티 시새는?   >>>   두산에너빌리티 시세는?   >>>   Intent: [('두산에너빌리티', 'st'), ('시세', 'pr')]
LG 우 종가는?   >>>   LG우 종가는?   >>>   Intent: [('종가', 'pr')]
